In [88]:
%load_ext autoreload
%autoreload 2
%pylab inline

import pyspiel
import open_spiel.python.examples.ubc_utils
from open_spiel.python.visualizations import ubc_treeviz
from open_spiel.python.examples.ubc_utils import *
from absl import logging
logging.set_verbosity(logging.INFO)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


/apps/open_spiel/venv/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['logging']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [89]:
### Load game
game = smart_load_sequential_game('clock_auction', game_parameters=dict(filename='parameters.json'))


In [90]:
solver = pyspiel.CFRSolver(game)
for i in range(1000):
    solver.evaluate_and_update_policy()
policy = solver.average_policy()

In [114]:
def make_policy_decorators(policy):
    def edge_weight_by_policy_decorator(parent_state, unused_child, action, **kwargs):
        attrs = ubc_treeviz.default_edge_decorator(parent_state, unused_child, action)  # get default attributes
        if parent_state.current_player() < 0:
            return attrs
        
        if '@ $0' in attrs['label']:
            attrs['label'] = 'Drop Out'
        
        action_prob = policy.action_probabilities(parent_state)[action]
        attrs['arrowsize'] = action_prob * attrs['arrowsize']
        attrs['penwidth'] = action_prob
        attrs['label'] = f'[{action_prob:.2f}] {attrs["label"]}'
        print(attrs, parent_state.current_player())
        return attrs

    def node_weight_by_policy_decorator(state, **kwargs):
        attrs = treeviz.default_node_decorator(state)
        if state.is_terminal():
            return_list = eval(attrs['label'])
            attrs['label'] = ', '.join([f'{x:.2f}' for x in return_list])
        
        # TODO: If terminal, report allocation
        # {'label': 'Current player: 0\np0v125, 125b150\n', 'fontsize': 8, 'width': 0.25, 'height': 0.25, 'margin': 0.01, 'shape': 'square', 'color': 'blue'}
        #         print(attrs)
        return attrs

    return node_weight_by_policy_decorator, edge_weight_by_policy_decorator

node_policy_decorator, edge_policy_decorator = make_policy_decorators(policy)

#   player = parent.current_player()
#   attrs = {
#       "label": " " + parent.action_to_string(player, action),
#       "fontsize": _FONTSIZE,
#       "arrowsize": _ARROWSIZE
#   }
#   attrs["color"] = _PLAYER_COLORS.get(player, "black")
#   return attrs


In [115]:
gametree = ubc_treeviz.GameTree(
    game,
    node_decorator=node_policy_decorator,
    edge_decorator=edge_policy_decorator,
    group_infosets=True,
    group_terminal=False,
    group_pubsets=False, 
    target_pubset='*',
#     state_prob_limit=0.01,
    action_prob_limit=0.01,
    policy=policy
)

outfile = './gametree.pdf'

# logging.info("Game tree:\n%s", gametree.to_string())

gametree.draw(outfile, prog='dot')
logging.info("Game tree saved to file: %s", outfile)


INFO:absl:Game tree saved to file: ./gametree.pdf


{'label': '[0.50]  Bid for 0,1 licenses @ $100 with activity 100', 'fontsize': 8, 'arrowsize': 0.2480732513899798, 'color': 'blue', 'penwidth': 0.4961465027799596} 0
{'label': '[1.00]  Bid for 0,1 licenses @ $100 with activity 100', 'fontsize': 8, 'arrowsize': 0.49954758119272386, 'color': 'red', 'penwidth': 0.9990951623854477} 1
{'label': '[1.00]  Bid for 1,0 licenses @ $100 with activity 100', 'fontsize': 8, 'arrowsize': 0.4997540629364291, 'color': 'blue', 'penwidth': 0.9995081258728582} 0
{'label': '[0.31] Drop Out', 'fontsize': 8, 'arrowsize': 0.1550257191874088, 'color': 'red', 'penwidth': 0.3100514383748176} 1
{'label': '[0.69]  Bid for 0,1 licenses @ $110 with activity 100', 'fontsize': 8, 'arrowsize': 0.344918674942753, 'color': 'red', 'penwidth': 0.689837349885506} 1
{'label': '[0.50]  Bid for 1,0 licenses @ $100 with activity 100', 'fontsize': 8, 'arrowsize': 0.25176008194335353, 'color': 'blue', 'penwidth': 0.5035201638867071} 0
{'label': '[1.00]  Bid for 0,1 licenses @ $10

In [ ]:
# TODO: Given a STRATEGY, colour the lines according to thickness?